In [9]:
from utils import list_images, read_image, extract_pixel_size, correct_pixel_size, make_isotropic, remove_labels_touching_edge, simulate_cytoplasm
from pathlib import Path
import napari
import pyclesperanto_prototype as cle
from skimage.measure import regionprops_table
import pandas as pd

cle.select_device("RTX")

<NVIDIA GeForce RTX 4090 on Platform: NVIDIA CUDA (2 refs)>

In [10]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("./raw_data/20250516LPSN120_C3neo_myc_rescue")

# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr),(..., ...)]
# Remember in Python one starts counting from 0, so your first channel will be 0
# i.e. markers = [("c3neo", 0), ("myc-tag", 1)]

markers = [("c3neo", 1), ("myc-tag", 2)]
nuclei_channel = 0

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
# Now, in addition to xy, you can downsample across your z-stack
slicing_factor_xy = 4 # Use 2 or 4 for downsampling in xy (None for lossless)
slicing_factor_z = None # Use 2 to select 1 out of every 2 z-slices

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['raw_data\\20250516LPSN120_C3neo_myc_rescue\\01.nd2',
 'raw_data\\20250516LPSN120_C3neo_myc_rescue\\02.nd2',
 'raw_data\\20250516LPSN120_C3neo_myc_rescue\\03.nd2',
 'raw_data\\20250516LPSN120_C3neo_myc_rescue\\04.nd2',
 'raw_data\\20250516LPSN120_C3neo_myc_rescue\\05.nd2',
 'raw_data\\20250516LPSN120_C3neo_myc_rescue\\06.nd2',
 'raw_data\\20250516LPSN120_C3neo_myc_rescue\\07.nd2',
 'raw_data\\20250516LPSN120_C3neo_myc_rescue\\08.nd2',
 'raw_data\\20250516LPSN120_C3neo_myc_rescue\\09.nd2',
 'raw_data\\20250516LPSN120_C3neo_myc_rescue\\10.nd2',
 'raw_data\\20250516LPSN120_C3neo_myc_rescue\\11.nd2']

In [11]:
image = images[1]

img, filename = read_image(image, slicing_factor_xy, slicing_factor_z)

# Obtain xyz pixel (voxel) size from image metadata (nd2 files)
voxel_size = extract_pixel_size(images[0])

# Correct pixel size if slicing is applied
pixel_size_x, pixel_size_y, voxel_size_z = correct_pixel_size(voxel_size, slicing_factor_xy, slicing_factor_z)

nuclei_resampled = make_isotropic(img[nuclei_channel], pixel_size_x, pixel_size_y, voxel_size_z)

# Remove background with a top_hat_filter
#background_subtracted = cle.top_hat_box(nuclei_resampled, radius_x=5, radius_y=5, radius_z=5)

# Apply gaussian blur to prevent the formation of holes upon labeling
post_gaussian = cle.gaussian_blur(nuclei_resampled, sigma_x=2, sigma_y=2, sigma_z=2)

# Voronoi-Otsu labeling
segmented = cle.voronoi_otsu_labeling(post_gaussian, spot_sigma=10, outline_sigma=1)

# Close holes in labels to avoid false emtpy volumes within the nuclei
segmented = cle.closing_labels(segmented, radius=5)

# Remove labels touching image xy borders
nuclei_labels = remove_labels_touching_edge(segmented.get())

# Simulate cytoplasm
cytoplasm_labels = simulate_cytoplasm(nuclei_labels, dilation_radius=10)



Image analyzed: 02
Original Array shape: (3, 31, 2720, 2720)
Compressed Array shape: (3, 31, 680, 680)


In [12]:
viewer = napari.Viewer(ndisplay=2)

viewer.add_image(nuclei_resampled)

for channel_name, ch_nr in markers:
    # Resample marker, transform into Numpy array and visualize
    marker_resampled = make_isotropic(img[ch_nr], pixel_size_x, pixel_size_y, voxel_size_z).get()
    viewer.add_image(marker_resampled, name=channel_name)

viewer.add_labels(nuclei_labels)
viewer.add_labels(cytoplasm_labels)

<Labels layer 'cytoplasm_labels' at 0x22632087ca0>

In [13]:
# Create an  empty list to hold the per channel info
props_list = []

# Create a dictionary containing all image descriptors
descriptor_dict = {"filename": filename, "folder_name": directory_path.name}

# Loop through each channel, resample it and extract the average intensity within nuclei and cytoplasm
for channel_name, ch_nr in markers:
    print(f"Analyzing channel: {channel_name}")

    # Resample marker and transform into numpy array
    marker_resampled = make_isotropic(img[ch_nr], pixel_size_x, pixel_size_y, voxel_size_z).get()

    # Extract intensity information from each marker channel
    nuclei_props = regionprops_table(label_image=nuclei_labels,
                            intensity_image=marker_resampled,
                            properties=["label", "intensity_mean"])
    
    # Extract intensity information from each marker channel
    cyto_props = regionprops_table(label_image=cytoplasm_labels,
                            intensity_image=marker_resampled,
                            properties=["label", "intensity_mean"])
    
    # Convert to dataframe
    nuclei_props_df = pd.DataFrame(nuclei_props)
    cyto_props_df = pd.DataFrame(cyto_props)

    # Rename intensity_mean column to indicate the specific image
    nuclei_props_df.rename(columns={"intensity_mean": f"nuclei_{channel_name}_avg_int"}, inplace=True)
    cyto_props_df.rename(columns={"intensity_mean": f"cyto_{channel_name}_avg_int"}, inplace=True)

    # Merge nuclei and cyto on label
    props_df = pd.merge(nuclei_props_df, cyto_props_df, on='label')

    # Append each props_df to props_list
    props_list.append(props_df)

# Initialize the df with the first df in the list
img_props_df = props_list[0]
# Start looping from the second df in the list
for df in props_list[1:]:
    img_props_df = img_props_df.merge(df, on="label")

# Add each key-value pair from descriptor_dict to props_df at the specified position
insertion_position = 0
for key, value in descriptor_dict.items():
    img_props_df.insert(insertion_position, key, value)
    insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

Analyzing channel: c3neo
Analyzing channel: myc-tag


In [14]:
img_props_df

,filename,folder_name,label,nuclei_c3neo_avg_int,cyto_c3neo_avg_int,nuclei_myc-tag_avg_int,cyto_myc-tag_avg_int
0,02,20250516LPSN120_C3neo_myc_rescue,1,1166.818237,1227.920532,1042.935669,1039.128662
1,02,20250516LPSN120_C3neo_myc_rescue,2,1146.658569,1229.524414,1032.202148,1029.705078
2,02,20250516LPSN120_C3neo_myc_rescue,3,1233.642456,1271.022217,1037.400024,1030.235718
3,02,20250516LPSN120_C3neo_myc_rescue,4,1146.475220,1215.136353,1033.381470,1029.056519
4,02,20250516LPSN120_C3neo_myc_rescue,5,1075.000000,1172.323486,953.000000,1009.823547
5,02,20250516LPSN120_C3neo_myc_rescue,6,1186.011475,1178.001343,1030.919189,1025.702759
6,02,20250516LPSN120_C3neo_myc_rescue,7,1237.833008,1304.070190,1029.732910,1030.874390
7,02,20250516LPSN120_C3neo_myc_rescue,8,1185.720215,1183.330078,1032.414429,1026.953491
8,02,20250516LPSN120_C3neo_myc_rescue,9,1423.292725,1330.804443,1227.413940,1117.740967
9,02,20250516LPSN120_C3neo_myc_rescue,10,1205.704102,1289.584106,1028.698853,1030.073975


In [15]:
#TODO: Extract folder_names, loop over them, then loop over each image inside them. Save .csv on a per folder name basis.